In [1]:
import pandas as pd
from datetime import datetime
import csv
import pickle

In [2]:
transaktion_data = pd.read_csv("data/transactions.csv", dtype=str)

In [3]:
input_file = "data/sebank_customers_with_accounts.csv"
output_pickle = "data/customers_cleaned.pkl"
log_file = "logging_folder/duplicate_personnummer.log"

seen_personnummer = set()
unique_rows = []
duplicates = []

with open(input_file, encoding='utf-8') as infile, \
     open(log_file, 'w', encoding='utf-8') as log:

    reader = csv.DictReader(infile)

    for row in reader:
        personnummer = row.get('Personnummer')

        if personnummer in seen_personnummer:
            log.write(",".join(row.values()) + "\n")
            duplicates.append(row)
        else:
            seen_personnummer.add(personnummer)
            unique_rows.append(row)

with open(output_pickle, 'wb') as pkl_file:
    pickle.dump(unique_rows, pkl_file)


In [4]:
df_transaction = pd.DataFrame(transaktion_data)
#df_transaction.dtypes

In [5]:
df_transaction["amount"] = df_transaction["amount"].str.replace(" ", "").str.replace(",", ".")
df_transaction["amount"] = df_transaction["amount"].astype(float)

In [6]:
#df_transaction["currency"].unique()
#df_transaction["sender_country"].unique()
#df_transaction["receiver_country"].unique()
#df_transaction["notes"].unique()

In [7]:
expected_formats = [
    "%Y%m%d %H:%M:%S",   # 20250125 04:48:00
    "%y-%m-%d %H:%M:%S", # 25-04-09 12:12:00
    "%Y-%m-%d %H:%M",    # 2025-01-18 16:14
    "%Y-%m-%d %H.%M",    # 2025-01-30 23.30
    "%Y-%m-%d %H.%M:%S"  # 2025-04-01 09.15:00
]
converted_data_with_ids = []
invalid_data_with_ids = []

try:
    with open ("logging_folder/timestamp_logs_with_ids", "r", encoding="utf-8") as file:
        for line in file:
            line = line.strip()
            if not line:
                continue

            parts = line.split(',', 1)
            if len(parts) < 2:
                invalid_data_with_ids.append((None, f"Formatfel i loggfilen: {line}"))
                continue

            original_id = parts[0]
            date_str = parts[1]

            dt_obj = None
            original_date_str_from_log = date_str

            next_day_adjustment = False
            if " 24" in date_str:
                date_str = date_str.replace(" 24", " 00", 1)
                next_day_adjustment = True

            if dt_obj and next_day_adjustment:
                dt_obj += pd.Timedelta(days=1)

            if "." in date_str.split(" ")[-1] and ":" not in date_str.split(" ")[-1]:
                 time_part = date_str.split(" ")[-1]
                 if len(time_part.split('.')) == 2: # Ex: 12.30
                     date_str = date_str.replace(time_part, time_part.replace('.', ':'))

            for formats in expected_formats:
                try:
                    dt_obj = datetime.strptime(date_str, formats)
                    break
                except ValueError:
                    continue

            if dt_obj is None:
                try:
                    dt_obj = pd.to_datetime(date_str, errors="raise")
                except ValueError as e:
                    invalid_data_with_ids.append((original_id, f"{original_date_str_from_log} - pandas hanterar inte detta: {e}"))
                    continue

            if dt_obj and next_day_adjustment:
                dt_obj += pd.Timedelta(days=1)

            if dt_obj:
                converted_data_with_ids.append((original_id, dt_obj.strftime("%Y-%m-%d %H:%M:%S")))

except FileNotFoundError:
    print("I searched for the file: 'timestamp_logs_with_ids' but it remains hidden")
except Exception as e:
    print(f"An error occurred: {e}")

print("\nIts converted! (YYYY-MM-DD HH:MM:SS):")
for original_id, converted_date in converted_data_with_ids:
    print(f"ID: {original_id}, Datum: {converted_date}")

print("\nUnknown error amongst id, file import or convertion error")
for original_id, error_msg in invalid_data_with_ids:
    print(f"ID: {original_id}, Fel: {error_msg}")


Its converted! (YYYY-MM-DD HH:MM:SS):

Unknown error amongst id, file import or convertion error


In [8]:
if "id" in df_transaction.columns:
    original_ids = df_transaction["id"].copy()
else:
    original_ids = df_transaction.index.copy()

original_timestamps = df_transaction["timestamp"].copy()
df_transaction["timestamp"] = pd.to_datetime(df_transaction["timestamp"], errors="coerce")

df_valid_timestamps = df_transaction[df_transaction["timestamp"].notna()]
df_pending = df_transaction[df_transaction["timestamp"].isna()]

In [9]:
pending_data_to_log = []
for idx in df_pending.index:
    item_id = original_ids[idx] if "id" in df_transaction.columns else idx
    pending_data_to_log.append((item_id, original_timestamps[idx]))

print(f"Pending data to log: {pending_data_to_log}")

Pending data to log: []


In [10]:
if not df_pending.empty:
    with open("logging_folder/timestamp_logs_with_ids", "a", encoding="utf-8") as f:
        for item_id, val in pending_data_to_log:
            f.write(f"{item_id},{val}")
    print("timestamp_logs_with_ids.txt har uppdaterats med ID:n.")
else:
    print("Inga felaktiga datum att logga.")


Inga felaktiga datum att logga.


In [11]:
id_to_converted_datetime = {item_id: dt_obj for item_id, dt_obj in converted_data_with_ids}

for idx in df_pending.index:
    current_id = original_ids[idx]

    if current_id in id_to_converted_datetime:
        df_transaction.loc[idx, "timestamp"] = id_to_converted_datetime[current_id]

print(df_transaction["timestamp"])

0       2025-01-08 03:17:00
1       2025-01-02 19:34:00
2       2025-01-12 20:08:00
3       2025-02-08 06:24:00
4       2025-03-01 18:51:00
                ...        
99995   2025-01-08 18:00:00
99996   2025-03-20 10:00:00
99997   2025-05-12 09:00:00
99998   2025-03-05 15:00:00
99999   2025-05-11 09:00:00
Name: timestamp, Length: 100000, dtype: datetime64[ns]


In [12]:
df_transaction.to_pickle("data/cleaned_transaction.pk1")